# DEVNET-2449 Python For the Enterprise Automation Exloration

# Network Device Programability

## What's it about
This class focuses on various aspects of *Network Device Programability*. In other words: How to use a *program or script* to configure and / or operate a network device (a router, a switch, a firewall). This is an alternative with various benefits to do the same thing in the traditional way, namely by using the device's CLI.

## Why This Matters
The following image illustrates why Network Device Programability matters

![](images/whyndpmatters.png)

While a lot of money is spent running and maintaining network infrastructure compared to the initial hardware spend, it takes considerable more time when it comes to deployment of new services compared to data center resources like *compute* or *storage*.

Programability solutions can help to bring down that time while, simultaneously, help with

- repeatability
- reliability 
- consistency

when operating network devices.

## High Level Overview

![](images/agents.png)


## Jupyter Notebooks
The *Jupyter Notebooks* in this workbench class are interactive learning environments. We are using them to learn about network device programability in combination with virtual Cisco network devices.

Notebooks are **interactive**, the code and text in each notebook can be changed on the fly and the result of those changes can be seen immediately within the notebook itself.

Notebooks are organized in *cells*, each displayed *rectangle* or *cell* is basically either Markdown text (like this cell) or Python code. Python code can be *executed* by either clicking the  <span class="inline">![](images/forwardicon.png)</span> in the 'Cell' menu in the menu bar or the key combination 'Shift-Enter'.

<div class="my-notify-warning">

<p>**IMPORTANT for Execution. PLEASE READ**. At its simplest, this is a playbook and execution is top-down. Code cells will look something like `In [#]:` (with '#' being a number). That's where you want to single-click. To execute the code, simply hit Shift+Enter.</p>

<p>The code should execute, and move onto the next cell. The number in `In [#]:` will change and if there is some return value (as opposed to an explicit `print()` statement) it will appear in an `Out [#]:` cell. The numbers will change after each cell execution.</p>

<p>It's easy to "execute" an entire notebook, task-by-task, by continuing to execute Shift+Return. Also, if you ever see `In [*]:` in your code cells (note the asterisk '\*'), it just means things are executing, and you are probably waiting on data to come back.</p>

<p>So depending on what you're doing, some patience might be required.</p>

</div>


## Environment
The workbench environment is running entirely inside of the provided MacBook Pros. We use [Vagrant](https://www.vagrantup.com/) to run two [CSR 1000V virtual routers](http://www.cisco.com/c/en/us/products/routers/cloud-services-router-1000v-series/index.html) running IOS XE 16.3.2 and we use [Docker](https://www.docker.com/) to provide the [Jupyter](http://jupyter.org/) interactive notebooks (like the document you are currently looking at).

The Python environment in the notebooks communicate with the routers via a host-only *internal* network on the MacBook Pro. The MacBook has its own IP address on this host-only network: `172.20.20.1`. The container can access the routers on their own IP addresses, `172.20.20.10` and `172.20.20.20` respectively. The following diagram shows a high level overview of the environment:

![Topology](images/topology.png)

We can access the CLI of the CSR1000v using a terminal (`Terminal.app`) and, while in the Vagrant directory, typing `vagrant ssh rtr1` or `vagrant ssh rtr2`. Because a public key has been installed into the router(s), we get a privileged shell / CLI without being prompted for a password.

While on the router, you can check the 'wsma' configuration section and also the remaining configuration which enables NETCONF and RESTCONF. 

Apart from a few static routes and an OSPF process there is not a lot else configured on the box.

In [ ]:
HOST = '172.20.20.10'
PORT = 830
USER = 'vagrant'
PASS = 'vagrant'

In [ ]:
import jtextfsm as textfsm
from netmiko import ConnectHandler
import pprint

def get_show_inventory(ip, username, password, enable_secret):
    # establish a connection to the device
    ssh_connection = ConnectHandler(
        device_type='cisco_ios',
        ip=ip,
        username=username,
        password=password,
        secret=enable_secret)
    # enter enable mode
    ssh_connection.enable()
    # prepend the command prompt to the result (used to identify the local device)
    result = ssh_connection.find_prompt() + "\n"
    # execute the show inventory command with extended delay
    result += ssh_connection.send_command("show inventory", delay_factor=2)
    # close SSH connection
    ssh_connection.disconnect()
    return result


In [ ]:
    raw_text_data = get_show_inventory(HOST, USER, PASS, PASS)
    print ('This is what the text looks like\n' + raw_text_data)
    # Run the text through the FSM. 
    # The argument 'template' is a file handle and 'raw_text_data' is a 
    # string with the content from the show_inventory.txt file
    template = open("/home/docker/cli/show_inventory_multiple.textfsm")
    re_table = textfsm.TextFSM(template)
    fsm_results = re_table.ParseText(raw_text_data)
    # build keys for inventory sub-items 
    keys = [s for s in re_table.header]
    #build inventory list of dictionaries
    inventory = []
    for row in fsm_results:
      entry = {}
      for i, item in enumerate(row):
        entry.update({keys[i]:item})
      inventory.append(entry)
    pp = pprint.PrettyPrinter(indent=2)

In [ ]:
    print(template.read())

In [ ]:
    print('Results translated through regular expression template organized as a dictonary')
    pp.pprint(inventory) # Print full inventory as dictionary
    print([key for key, value in inventory[0].items()])  # print key values as list
    for item in inventory:   # print inventory elements a table 
        print([item[key] for key in item])
    

In [ ]:
    sought_product = 'CSR'
    resultlist = [d for d in inventory if d.get('productid', '') == sought_product]
    print('\nFound Devices with specific productid ' + sought_product)
    pp.pprint(resultlist) #print list of devices with sought productid
    #print ([(item['hostname'], item['productid']) for item in resultlist])  #print found items as a tuple

In [ ]:
def get_show_ip_route(ip, username, password, enable_secret):
    # establish a connection to the device
    ssh_connection = ConnectHandler(
        device_type='cisco_ios',
        ip=ip,
        username=username,
        password=password,
        secret=enable_secret)
    # enter enable mode
    ssh_connection.enable()
    # prepend the command prompt to the result (used to identify the local device)
    result = ssh_connection.find_prompt() + "\n"
    # execute the show inventory command with extended delay
    result += ssh_connection.send_command("show ip route", delay_factor=2)
    # close SSH connection
    ssh_connection.disconnect()
    return result

In [ ]:
raw_text_data = get_show_ip_route(HOST, USER, PASS, PASS)
print ('This is what the text looks like\n' + raw_text_data)

In [ ]:
import ncclient.manager
import xml.dom.minidom
from ncclient.operations import TimeoutExpiredError
  
route_filter = '''
  <routing-state xmlns="urn:ietf:params:xml:ns:yang:ietf-routing">
    <routing-instance>
      <ribs>
        <rib>
          <name>ipv4-default</name>
          <address-family xmlns:v4ur="urn:ietf:params:xml:ns:yang:ietf-ipv4-unicast-routing">v4ur:ipv4-unicast</address-family>
          <default-rib>true</default-rib>
          <routes/>
        </rib>
      </ribs>
    </routing-instance>
  </routing-state>
  '''
nckwargs = dict(
    host=HOST,
    port=PORT,
    hostkey_verify=False,
    username=USER,
    password=PASS,
    device_params={'name':"csr"}
)
m = ncclient.manager.connect(**nckwargs)  
try:
    print ('Here we are printing the RIB as XML\n')
    c = m.get(filter=('subtree', route_filter))
    xmlDom = xml.dom.minidom.parseString(str(c))
    print (xmlDom.toprettyxml( indent = " " ))
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("ERORR severity={}, tag={}".format(e.severity, e.tag))

In [ ]:
ns = 'urn:ietf:params:xml:ns:yang:ietf-routing'
nsmap = dict(rt=ns)
route_table=[]
routes = c.data.findall(".//{%s}route" % ns)
for i, route in enumerate(routes):
    dest = route.xpath('./rt:destination-prefix/text()', namespaces=nsmap)
    hop = route.xpath('./rt:next-hop/rt:next-hop-address/text()', namespaces=nsmap)
    intfc = route.xpath('./rt:next-hop/rt:outgoing-interface/text()', namespaces=nsmap)
    prot = route.xpath('./rt:source-protocol/text()', namespaces=nsmap)
    if len(hop) > 0:
        entry = {'hop':hop[0]}
    if len(intfc) > 0:
        entry = {'intfc':intfc[0]}  
    entry.update({'dest':dest[0], 'prot':prot[0]}) 
    route_table.append(entry)
print ('\nOr we can just grab the information that is interesting\n')
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(route_table)  

In [ ]:
cpu_util = '''<filter>
<cpu-usage xmlns="urn:cisco:params:xml:ns:yang:cisco-process-cpu">
<cpu-utilization>
</cpu-utilization>
</cpu-usage></filter>'''

cpu_process = '''<filter>
<cpu-usage xmlns="urn:cisco:params:xml:ns:yang:cisco-process-cpu" xmlns:y="http://tail-f.com/ns/rest" xmlns:pcpu="urn:cisco:params:xml:ns:yang:cisco-process-cpu">
<process-cpu-usage>
</process>
</process-cpu-usage>
</filter>'''
print ('Using NETCONF, you can present data as XML without any conversion')
cpu_util = str( m.get( filter=cpu_util ) )
xmlDom = xml.dom.minidom.parseString(cpu_util)
print xmlDom.toprettyxml( indent = " " )

In [ ]:
import xmltodict
print ('Or you can convert to Dictionary')
cpu_dict = xmltodict.parse( cpu_util )['rpc-reply']['data']
print [(i,v) for i,v in cpu_dict['cpu-usage']['cpu-utilization'].iteritems()]

In [ ]:
print ('\nWith structured data you can filter or sort \nHere we are grabbing the processes with higher run-time')
cpu_dict = xmltodict.parse( str( m.get( filter=cpu_process ) ) )
cpu_dict = cpu_dict['rpc-reply']['data']
nonzero_process = [t for t in cpu_dict['cpu-usage']['process-cpu-usage']['process'] if int(t['total-run-time']) > 100]
for i in nonzero_process: print (i['name'],i['pid'],i['total-run-time'])

In [ ]:
CONF_FILTER ='''<filter>
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface></interface>
  </interfaces>
</filter>'''
STATE_FILTER ='''<filter>
  <interfaces-state xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface></interface>
  </interfaces-state>
</filter>'''
print('\nHere is the XML data for Interfaces Operational State')
interfaces = m.get(STATE_FILTER)
pp = pprint.PrettyPrinter(indent=2)
print(xml.dom.minidom.parseString(interfaces.xml).toprettyxml(indent=" "))

In [ ]:
m = ncclient.manager.connect(**nckwargs)
print('\nHere is the XML data for Interfaces Configuration')
interfaces = m.get_config('running',CONF_FILTER)
print(xml.dom.minidom.parseString(interfaces.xml).toprettyxml(indent=" "))

In [ ]:
from subprocess import Popen, PIPE, STDOUT
SCHEMA_TO_GET = 'ietf-interfaces'
print(' We are connecting to the device and downloading the YANG Module')
print("Now we will do a 'pyang -f tree of %s.yang'" % SCHEMA_TO_GET)
print('This structure is what data we can get (ro/rw) or set (rw)')
c = m.get_schema(SCHEMA_TO_GET)
p = Popen(['pyang', '-f', 'tree'], stdout=PIPE, stdin=PIPE, stderr=PIPE)
stdout_data = p.communicate(input=c.data)[0]
print stdout_data

In [ ]:
from ydk.services import CRUDService
from ydk.providers import NetconfServiceProvider
from ydk.models.ietf import ietf_interfaces as intf
from ydk.models.ietf import iana_if_type as iftype

# create NETCONF provider
provider = NetconfServiceProvider(address=HOST,
                                  port=PORT,
                                  username=USER,
                                  password=PASS)

# create CRUD service
crud = CRUDService()

# query object
q_i = intf.Interfaces()

# get stuff
intfs = crud.read(provider, q_i)

# print interface names and types
for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))

In [ ]:
q_i = intf.InterfacesState()

# get stuff
intfs = crud.read(provider, q_i)
q_i = intf.InterfacesState()

# get stuff
intfs = crud.read(provider, q_i)
int_info=[(i.name, i.statistics.out_pkts, int(i.speed)/1000000, i.oper_status) for i in intfs.interface]
for thing in int_info: print(str(thing))

# Create Software Loopback

In [ ]:
# new interface
new_loopback = intf.Interfaces.Interface()

# create a new loopback
new_loopback.name = 'Loopback666'
new_loopback.type = iftype.SoftwareloopbackIdentity()
new_loopback.description = 'Created by DevNet2445'
res = crud.create(provider, new_loopback)

In [ ]:
# get stuff
q_i = intf.Interfaces()
intfs = crud.read(provider, q_i)

for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))

# Delete Software Loopback

In [ ]:
# create CRUD service
crud = CRUDService()

# interface to delete
to_del = intf.Interfaces.Interface()

# create a new loopback
to_del.name = 'Loopback666'
res = crud.delete(provider, to_del)

In [ ]:
# get stuff
intfs = crud.read(provider, q_i)

for i in intfs.interface:
    print('%s, %s, %s' % (i.name, i.type._meta_info().yang_name, i.description))
provider.close()